# DISPONIBILIZACIÓN DEL MODELO

(30 puntos)

Se disponibiliza el modelo en una API alojada en un servicio en la nube. (20 puntos)                                                                           
Se hacen las predicciones sobre el valor del automóvil en al menos dos observaciones del set de validación. (10 puntos)

In [1]:
# Importação de bibliotecas
from flask import Flask, request, jsonify
from flask_restx import Api, Resource, fields
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from werkzeug.utils import cached_property
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import numpy as np
import joblib
import traceback
import nltk
import string

# Carregar o modelo, vetor TF-IDF e MultiLabelBinarizer
model = joblib.load('modelo.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')
mlb = joblib.load('mlb.pkl')

# Crear la aplicación Flask con el nombre "api_grupo5"
app = Flask("api_grupo5")

# Definir la API Flask con Flask-Restx
api = Api(
    app,
    version='1.0',
    title='Movie Genre Classification API',
    description='Classify the genre of a movie based on its features'
)

# Faça o download dos recursos necessários do NLTK (se ainda não tiver feito)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Função de pré-processamento de texto
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation and not word.isdigit()]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    processed_text = ' '.join(tokens)
    return processed_text

movie_genre_model = api.model('MovieGenreModel', {
    'plot': fields.String(required=True, description='Plot of the movie')
})

# Definir la ruta para la API
@api.route('/classify')
class MovieGenreClassification(Resource):
    @api.expect(movie_genre_model)
    def post(self):
        try:
            # Obtener los datos de entrada
            data = request.json
            print("Datos recibidos:", data)
            X_input = pd.DataFrame([data])
            print("DataFrame de entrada:", X_input)

            # Aplicar el preprocesamiento del texto
            processed_input = X_input['plot'].apply(preprocess_text)
            print("Texto procesado:", processed_input)

            # Vectorizar el texto preprocesado
            vectorized_text = vectorizer.transform(processed_input)
            print("Texto vectorizado:", vectorized_text)

            # Realizar la predicción directamente con el modelo
            prediction = model.predict(vectorized_text)
            print("Predicción:", prediction)

            # Convertir la predicción a nombres de géneros
            genre_predictions = mlb.inverse_transform(prediction)
            print("Géneros predecidos:", genre_predictions)

            # Devolver la predicción
            return {'prediction': genre_predictions}

        except Exception as e:
            traceback.print_exc()
            return {'error': str(e)}, 500

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

 * Serving Flask app 'api_grupo5'
 * Debug mode: on


[nltk_data] Downloading package punkt to /home/felipe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/felipe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/felipe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/felipe/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.15.10:5000
Press CTRL+C to quit
127.0.0.1 - - [22/May/2024 13:34:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2024 13:34:29] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
127.0.0.1 - - [22/May/2024 13:34:29] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [22/May/2024 13:34:29] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [22/May/2024 13:34:29] "GET /swaggerui/swagge